In [1]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, ZeroPadding2D)
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import InputLayer
from tensorflow_core.python.keras.layers import  MaxPooling2D, Softmax
from tqdm.notebook import tqdm

In [2]:
def make_conv(in_size, out_size, **params):
    assert len(params) == 2, "Only kernel_size and filters are allowed in **params"
    kernel_size = params['kernel_size']
    filters = params['filters']
    w_i = in_size
    w_o = out_size

    tp = w_o - w_i - 1 + kernel_size
    p1 = tp // 2
    p2 = tp - p1
    assert w_o == w_i + 1 - kernel_size + p1 + p2
    block = [
        Conv2D(filters, kernel_size, ),
        ZeroPadding2D(((p1, p2), (p1, p2)))
    ]
    return block

def create_model(in_size, kernel_size, initial_filters):
    model = Sequential([
        InputLayer(input_shape=(in_size, in_size, 3)),
        *make_conv(in_size, out_size=in_size, filters=initial_filters, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//2, out_size = in_size//2, filters=initial_filters*2, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//4, out_size = in_size//4, filters=initial_filters*4, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        *make_conv(in_size//8, out_size = in_size//8, filters=initial_filters*8, kernel_size=kernel_size),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Softmax()
    ])

    return model


In [3]:
models = {}
for kernel_size in  [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    name = f'kernel_size_{kernel_size}_{kernel_size}'
    model = create_model(in_size=256, initial_filters=8, kernel_size=kernel_size)
    models[name] = model

In [4]:
# Print models
for name, model in models.items():
    print('-'*80)
    print('NAME ' + name)
    model.summary()


--------------------------------------------------------------------------------
NAME kernel_size_1_1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       32        
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      144       
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
____________________

## Convert models

In [7]:
# Convert the models
models_path = 'tflite_models/kernels'
os.makedirs(models_path, exist_ok=True)

for name, model in tqdm(models.items()):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # write it to file
    filepath = os.path.join(models_path, name + '.tflite')
    with open(filepath, 'wb') as f:
        f.write(tflite_model)

100%|██████████| 9/9 [00:23<00:00,  2.62s/it]


### Profile models

In [12]:
output_dir = 'results/kernels'
!python profiler.py --models-dir {models_path} --output-dir {output_dir}

0%|                                                     | 0/9 [00:00<?, ?it/s]Running: /home/loveml/Downloads/tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/loveml/Desktop/dsproject/tflite_models/kernels/kernel_size_5_5.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/loveml/Desktop/dsproject/results/kernels/kernel_size_5_5
STARTING!
Duplicate flags: num_threads
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/loveml/Desktop/dsproject/tflite_models/kernels/kernel_size_5_5.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to export profiling data to: [/home/loveml/Deskto

## Convert Models + Quantize

In [5]:
# Converting the models to tflite format and saving them
from tqdm import tqdm
models_path = "tflite_models/kernels_quant"
os.makedirs(models_path, exist_ok=True)

for name, model in tqdm(models.items()):
    name += '_quant'
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.compat.v1.lite.constants.FLOAT16]
    tflite_model = converter.convert()

    # write it to file
    filepath = f'{models_path}/{name}.tflite'
    with open(filepath, 'wb') as f:
        f.write(tflite_model)
    del tflite_model

100%|██████████| 9/9 [00:23<00:00,  2.62s/it]


### Profile models

In [6]:
output_dir = 'results/kernels_quant'
!python profiler.py --models-dir {models_path} --output-dir {output_dir}

0%|                                                     | 0/9 [00:00<?, ?it/s]Running: /home/loveml/Downloads/tensorflow/bazel-bin/tensorflow/lite/tools/benchmark/benchmark_model \
--graph=/home/loveml/Desktop/dsproject/tflite_models/kernels_quant/kernel_size_7_7_quant.tflite \
--enable_op_profiling=true \
--profiling_output_csv_file=/home/loveml/Desktop/dsproject/results/kernels_quant/kernel_size_7_7_quant
STARTING!
Duplicate flags: num_threads
Min num runs: [50]
Min runs duration (seconds): [1]
Max runs duration (seconds): [150]
Inter-run delay (seconds): [-1]
Num threads: [1]
Benchmark name: []
Output prefix: []
Min warmup runs: [1]
Min warmup runs duration (seconds): [0.5]
Graph: [/home/loveml/Desktop/dsproject/tflite_models/kernels_quant/kernel_size_7_7_quant.tflite]
Input layers: []
Input shapes: []
Input value ranges: []
Input layer values files: []
Allow fp16 : [0]
Require full delegation : [0]
Enable op profiling: [1]
Max profiling buffer entries: [1024]
CSV File to export pro